In [1]:
!npm install localtunnel


up to date, audited 23 packages in 680ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [2]:
!pip install streamlit

In [3]:
%%writefile my_streamlit_app.py

import streamlit as st
from sklearn import preprocessing
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np

# Load dataset
data = pd.read_csv('Expresso_churn_dataset.csv')
data1 = data.copy()

# Data preprocessing
data.drop(['TIGO', 'ZONE2', 'ZONE1', 'DATA_VOLUME', 'user_id', 'TOP_PACK', 'FREQ_TOP_PACK', 'ORANGE', 'MRG'], axis=1, inplace=True)
data['MONTANT'].fillna(data['MONTANT'].median(), inplace=True)
data['FREQUENCE_RECH'].fillna(data['FREQUENCE_RECH'].median(), inplace=True)
data['REVENUE'].fillna(data['REVENUE'].median(), inplace=True)
data['ARPU_SEGMENT'].fillna(data['ARPU_SEGMENT'].median(), inplace=True)
data['FREQUENCE'].fillna(data['FREQUENCE'].median(), inplace=True)
data['ON_NET'].fillna(data['ON_NET'].median(), inplace=True)

# Label encoding
label_encoder = preprocessing.LabelEncoder()
data['REGION'] = label_encoder.fit_transform(data['REGION'])
mapping_dict_region = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
data['TENURE'] = label_encoder.fit_transform(data['TENURE'])
mapping_dict_tenure = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Logistic Regression Model
x = data.drop(['CHURN'], axis=1)
y = data['CHURN']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
logreg = LogisticRegression()
logreg.fit(x_train, y_train)

# Streamlit page configuration
st.set_page_config(page_title="Churn Prediction App", page_icon="💻", layout="wide")

# Custom HTML & CSS for enhanced decoration and colors
st.markdown("""
    <style>
    /* Background Gradient */
    .stApp {
        background-image: linear-gradient(to right, #FFDEE9, #B5FFFC);
        color: #333333;
    }

    /* Title Styling */
    h1 {
        color: #FF6347;
        text-align: center;
        font-family: 'Comic Sans MS', cursive, sans-serif;
        font-size: 48px;
        text-shadow: 2px 2px #FF1493;
    }

    /* Button Styling */
    .stButton button {
        background-color: #FF69B4;
        color: white;
        font-size: 18px;
        padding: 10px 20px;
        border-radius: 10px;
        border: 2px solid #FF1493;
    }

    /* Input boxes */
    .stSelectbox, .stNumberInput {
        background-color: #FFFACD;
        border: 2px solid #FF69B4;
        border-radius: 5px;
        padding: 5px;
        color: #333333;
        font-size: 16px;
    }

    /* Prediction output styling */
    .churn-yes {
        color: #FF4500;
        font-size: 36px;
        font-weight: bold;
        text-align: center;
        background-color: #FFDEAD;
        padding: 10px;
        border-radius: 10px;
    }

    .churn-no {
        color: #32CD32;
        font-size: 36px;
        font-weight: bold;
        text-align: center;
        background-color: #98FB98;
        padding: 10px;
        border-radius: 10px;
    }

    </style>
""", unsafe_allow_html=True)

# Title with Emoji 🎉
st.markdown("<h1>Churn Prediction App 💻🎉</h1>", unsafe_allow_html=True)

# User options
choice = st.selectbox('Choose an option:', ('📊 Visualize', '🔮 Predict'))

if choice == '🔮 Predict':
    # Inputs for prediction with emojis 🎯
    st.markdown("<h3 style='color:#FF69B4;'>🎯 Enter Customer Details for Prediction:</h3>", unsafe_allow_html=True)

    region = st.selectbox('🌍 Select your region', data1['REGION'].unique())
    region_map = mapping_dict_region[region]
    TENURE = st.selectbox('⏳ Select your tenure', data1['TENURE'].unique())
    tenure_map = mapping_dict_tenure[TENURE]
    MONTANT = st.number_input('💰 Enter Montant', min_value=0.0, step=0.01)
    FREQUENCE = st.number_input('🔄 Enter FREQUENCE', min_value=0.0, step=0.01)
    REVENUE = st.number_input('📈 Enter REVENUE', min_value=0.0, step=0.01)
    ARPU_SEGMENT = st.number_input('📊 Enter ARPU_SEGMENT', min_value=0.0, step=0.01)
    FREQUENCE_RECH = st.number_input('🔄 Enter FREQUENCE_RECH', min_value=0.0, step=0.01)
    ON_NET = st.number_input('📶 Enter ON_NET', min_value=0.0, step=0.01)
    REGULARITY = st.number_input('🔁 Enter Regularity', min_value=0.0, step=0.01)


    my_dict0 ={'REGION': region, 'TENURE' : TENURE , 'MONTANT' : MONTANT,
         'FREQUENCE_RECH': FREQUENCE_RECH , 'REVENUE' : REVENUE,
          'ARPU_SEGMENT': ARPU_SEGMENT , 'FREQUENCE':FREQUENCE,
          'ON_NET': ON_NET , 'REGULARITY':REGULARITY}
    my_dict= {'REGION': region_map,
        'TENURE': tenure_map,
        'MONTANT': MONTANT,
        'FREQUENCE_RECH': FREQUENCE_RECH,
        'REVENUE': REVENUE,
        'ARPU_SEGMENT': ARPU_SEGMENT,
        'FREQUENCE':FREQUENCE,
        'ON_NET': ON_NET,
        'REGULARITY': REGULARITY}

    data_test= pd.DataFrame(my_dict, index = [0])
    data_test0 = pd.DataFrame(my_dict0, index = [0])



    # Display the entered data with emoji 📝
    if st.button('📝 Show My Entry'):
        st.write(data_test0)

    # Prediction output with emoji 🔮
    if st.button('🔮 Predict'):
        y_pred = (logreg.predict_proba(data_test)[:, 1] >= 0.7).astype(int)
        if y_pred == 0:
            st.markdown("<div class='churn-no'>🚫 No Churn</div>", unsafe_allow_html=True)
        else:
            st.markdown("<div class='churn-yes'>⚠️ Yes, Churn</div>", unsafe_allow_html=True)


Overwriting my_streamlit_app.py


In [4]:
!streamlit run my_streamlit_app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

your url is: https://wild-eyes-guess.loca.lt
